In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

movies_df = pd.read_csv('./dataset/movies.csv')
ratings_df = pd.read_csv('./dataset/ratings.csv')
tags_df = pd.read_csv('./dataset/tags.csv')

#Display Data Samp
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
#as we can see the tables are related to each other through common keys
#These tables are likely related to each other through movieIds or userIds 
#So we will combine the tables into a single table

# Merge tags_df with ratings_df based on movieId and userId
merged_df = pd.merge(tags_df, ratings_df, on=['movieId', 'userId'], how='left')

merged_df.head()

,userId,movieId,tag,timestamp_x,rating,timestamp_y
0,2,60756,funny,1445714994,5.0,1.445715e+09
1,2,60756,Highly quotable,1445714996,5.0,1.445715e+09
2,2,60756,will ferrell,1445714992,5.0,1.445715e+09
3,2,89774,Boxing story,1445715207,5.0,1.445715e+09
4,2,89774,MMA,1445715200,5.0,1.445715e+09


In [11]:
combined_df = pd.merge(merged_df, movies_df, on='movieId', how='left')
combined_df.head()

,userId,movieId,tag,timestamp_x,rating,timestamp_y,title,genres
0,2,60756,funny,1445714994,5.0,1.445715e+09,Step Brothers (2008),Comedy
1,2,60756,Highly quotable,1445714996,5.0,1.445715e+09,Step Brothers (2008),Comedy
2,2,60756,will ferrell,1445714992,5.0,1.445715e+09,Step Brothers (2008),Comedy
3,2,89774,Boxing story,1445715207,5.0,1.445715e+09,Warrior (2011),Drama
4,2,89774,MMA,1445715200,5.0,1.445715e+09,Warrior (2011),Drama


In [12]:
combined_df.to_csv("combined1.csv", index=False)

In [13]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   userId       3683 non-null   int64  
 1   movieId      3683 non-null   int64  
 2   tag          3683 non-null   object 
 3   timestamp_x  3683 non-null   int64  
 4   rating       3476 non-null   float64
 5   timestamp_y  3476 non-null   float64
 6   title        3683 non-null   object 
 7   genres       3683 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 230.3+ KB


In [14]:
combined_df.rename(columns={"timestamp_x":"tag_timestamp"},inplace=True)
combined_df.rename(columns={"timestamp_y":"rating_timestamp"},inplace=True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   userId            3683 non-null   int64  
 1   movieId           3683 non-null   int64  
 2   tag               3683 non-null   object 
 3   tag_timestamp     3683 non-null   int64  
 4   rating            3476 non-null   float64
 5   rating_timestamp  3476 non-null   float64
 6   title             3683 non-null   object 
 7   genres            3683 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 230.3+ KB


In [15]:
combined_df['tag_timestamp']=pd.to_datetime(combined_df['tag_timestamp'],unit='s')
combined_df['rating_timestamp']=pd.to_datetime(combined_df['rating_timestamp'],unit='s')
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   userId            3683 non-null   int64         
 1   movieId           3683 non-null   int64         
 2   tag               3683 non-null   object        
 3   tag_timestamp     3683 non-null   datetime64[ns]
 4   rating            3476 non-null   float64       
 5   rating_timestamp  3476 non-null   datetime64[ns]
 6   title             3683 non-null   object        
 7   genres            3683 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 230.3+ KB


In [18]:
print(combined_df.duplicated().sum())

0


In [19]:
combined_df.isna().sum()

userId                0
movieId               0
tag                   0
tag_timestamp         0
rating              207
rating_timestamp    207
title                 0
genres                0
dtype: int64

In [21]:
Q1=combined_df['rating'].quantile(.25)
Q3=combined_df['rating'].quantile(.75)
IQR=Q3-Q1 #range of the middle 50%
IQR

1.5

In [22]:
LV=Q1-(1.5*IQR)
UV=Q3+(1.5*IQR)
LV,UV

(1.25, 7.25)

In [23]:
combined_df['rating'].describe()

count    3476.000000
mean        4.016830
std         0.856925
min         0.500000
25%         3.500000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64